In [1]:
import numpy as np
from pydgrid import grid

In [27]:
Z_red = 110.0e3**2/5000.0e6
ratio=0.1
# Z**2 = R**2 + X**2
# Z**2 = ratio**2*X**2 + X**2 = X**2 * (ratio**2 + 1.0)
X_red = (Z_red**2/(ratio**2 + 1.0))**0.5
R_red = ratio*X_red

Z_trafo_b = 20.0e3**2/25.0e6
V_hv = 110.0/np.sqrt(3)*1.0

S1,pf1,S2,pf2 = 15300.0,0.98,5280,0.95
S_12_cplx = S1*pf1 + 1j*np.sin(np.arccos(pf1))*S1 + S2*pf2 + 1j*np.sin(np.arccos(pf2))*S2
pf_12 = np.cos(np.angle(S_12_cplx))

S1,pf1 = 40.0,0.85
S_13_cplx = S1*pf1 + 1j*np.sin(np.arccos(pf1))*S1
pf_13 = np.cos(np.angle(S_13_cplx))

S1,pf1,S2,pf2 = 215.0,0.97,390.0,0.85
S_14_cplx = S1*pf1 + 1j*np.sin(np.arccos(pf1))*S1 + S2*pf2 + 1j*np.sin(np.arccos(pf2))*S2
pf_14 = np.cos(np.angle(S_14_cplx))

data = {     
        "buses":[
                 {"bus": "HVinf", "pos_x":   0, "pos_y":  0, "units": "m", "U_kV":110.0},
                 {"bus": "HV",    "pos_x":   0, "pos_y": -5, "units": "m", "U_kV":110.0},
                 {"bus": "12",    "pos_x":   0, "pos_y":-15, "units": "m", "U_kV":20.0},
                 {"bus": "13",    "pos_x":   0, "pos_y":-35, "units": "m", "U_kV":20.0},
                 {"bus": "14",    "pos_x":   0, "pos_y":-85, "units": "m", "U_kV":20.0},
                ],
        "grid_formers":[
                        {"bus": "HVinf",
                        "bus_nodes": [1, 2, 3], "deg": [0, -120, -240],
                        "kV": [V_hv,V_hv,V_hv]}
                       ],
        "transformers":[
                        {"bus_j": "HV",  "bus_k": "12",  "S_n_kVA": 25.0e3, "U_j_kV":110, "U_k_kV":20.0,
                         "R_cc_pu": 0.001, "X_cc_pu":0.12, "connection": "Dyn1",   "conductors_j": 3, "conductors_k": 4}
                       ],
        "lines":[
                {"bus_j": "HVinf",  "bus_k": "HV",  "code": "eq_red", "m": 1000.0},
                {"bus_j": "12",     "bus_k": "13",  "code": "1", "m": 4.89e3},
                {"bus_j": "13",     "bus_k": "14",  "code": "1", "m": 3.0e3},
                ],
        "shunts":[
                 {"bus": "12" , "R": 0.001,   "X": 0.0, "bus_nodes": [4,0]},
                 {"bus": "14" , "R": 1.0e-10, "X": 0.0, "bus_nodes": [1,0]},
                 #{"bus": "12" , "R": 1.0e-10, "X": 0.0, "bus_nodes": [2,3]},
                 ],
        "loads":[
                # {"bus": "12" , "kVA":0*S_12_cplx, "pf": pf_12,"type":"3P+N"},
                # {"bus": "13" , "kVA":0*40.0, "pf": 0.85,"type":"3P+N"},
                # {"bus": "14" , "kVA":0*S_14_cplx, "pf":pf_14,"type":"3P+N"},
                ],
        "line_codes":
            {
            "eq_red":  {"R1":R_red,"X1":X_red},    
            "1":  {"R1":0.510,"X1":0.366,"B1_mu":3.172,"R0":0.658,"X0":1.611,"B0_mu":1.28},
            "2":  {"R1":89.0e-3,"X1":3.5e-3},
            }
       } 

In [29]:
sys1 = grid()
sys1.read(data)  # Load data
sys1.pf()  # solve power flow

from pydgrid.plot_bokeh import plot_results
fig = plot_results(sys1)

In [45]:
sys1 = grid()

fault_list = [('12','3ph'),('12','2ph'),('12','1ph-gnd'),
              ('13','3ph'),('13','2ph'),('13','1ph-gnd'),
              ('14','3ph'),('14','2ph'),('14','1ph-gnd'),
             ]

for bus,sc_type in fault_list:
    data["shunts"] = [
                     {"bus": "12" , "R": 0.001,   "X": 0.0, "bus_nodes": [4,0]}
                     ]
    if sc_type == '2ph':
        data["shunts"].append({"bus": bus , "R": 1.0e-10, "X": 0.0, "bus_nodes": [1,2]})
    if sc_type == '3ph':
        data["shunts"].append({"bus": bus , "R": 1.0e-10, "X": 0.0, "bus_nodes": [1,2]})
        data["shunts"].append({"bus": bus , "R": 1.0e-10, "X": 0.0, "bus_nodes": [2,3]})
    if sc_type == '1ph-gnd':
        data["shunts"].append({"bus": bus , "R": 1.0e-10, "X": 0.0, "bus_nodes": [1,0]})
    sys1.read(data)  # Load data    
    sys1.pf()  # solve power flow
    i_2a_m = sys1.transformers[0]['i_2a_m']
    print('{:s} fault in bus {:s} with current = {:0.4} kA'.format(sc_type,bus,i_2a_m/1000))

3ph fault in bus 12 with current = 5.774 kA
2ph fault in bus 12 with current = 5.0 kA
1ph-gnd fault in bus 12 with current = 5.852 kA
3ph fault in bus 13 with current = 2.538 kA
2ph fault in bus 13 with current = 2.198 kA
1ph-gnd fault in bus 13 with current = 1.8 kA
3ph fault in bus 14 with current = 1.82 kA
2ph fault in bus 14 with current = 1.576 kA
1ph-gnd fault in bus 14 with current = 1.246 kA


In [90]:
Z_trafo = 0.016+1j*1.92
I_cc_12 = 20.0e3/np.sqrt(3)/Z_trafo
Z_base_trafo = 20.0e3**2/25.0e6
Z_trafo_pu = Z_trafo/Z_base_trafo

In [44]:
abs(I_cc_12)

6013.856493222156

In [40]:
Z_trafo_pu

(0.001+0.12j)

In [23]:
sys1.transformers

[{'bus_j': 'HV',
  'bus_k': '12',
  'S_n_kVA': 25000.0,
  'U_j_kV': 110,
  'U_k_kV': 20.0,
  'R_cc_pu': 0.001,
  'X_cc_pu': 0.12,
  'connection': 'Dyn1',
  'conductors_j': 3,
  'conductors_k': 4,
  'bus_j_nodes': [1, 2, 3],
  'bus_k_nodes': [1, 2, 3, 4],
  'i_1a_m': 130.78245955787384,
  'i_1b_m': 130.82859976225936,
  'i_1c_m': 0.05254490739748195,
  'i_1n_m': 8.589309868000576e-14,
  'i_2a_m': 1246.148968529074,
  'i_2b_m': 0.29353147508296223,
  'i_2c_m': 0.27877781103658683,
  'i_2n_m': 1246.2668990840489,
  'deg_1a': -31.40554467746528,
  'deg_1b': 148.58344317926708,
  'deg_1c': -59.99585334038418,
  'deg_1n': -41.436731937093825,
  'deg_2a': 148.59473232891122,
  'deg_2b': -88.1864800606753,
  'deg_2c': 149.8327823062106,
  'deg_2n': -31.39370121460905}]

In [37]:
5774.2345111669765*1.1

6351.657962283674

In [49]:
5000*1.1

5500.0